# QUIZ - 2024-08

In [2]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import json
from service.models import *
from django.db.models import *
from django.db.models.functions import *
from django.utils import timezone
from datetime import datetime, date
from decimal import Decimal

## PART 1 - CRUD Operations
1.1 สร้างนัดหมาย (`Appointment`) ของลูกค้า "Customer 1" ซึ่งต้องการจองบริการ "Haircut" ของ "Provider 2"

โดยสร้างนัดหมายในวันที่ 2024-10-01 เวลา 11:00 น. 

(0.5 คะแนน)

*หมายเหตุ: จะต้อง get ข้อมูลมาโดยใช้การ query ด้วยชื่อตามที่โจทย์ว่าเท่านั้น ห้ามใช้ id ใน database*

In [8]:
customer_1 = Customer.objects.get(name = 'Customer 1')
provider_2 = ServiceProvider.objects.get(name = 'Provider 2')
haircut = Service.objects.get(service_provider = provider_2, name = 'Haircut')
appointment_1 = Appointment.objects.create(
    customer = customer_1,
    service = haircut,
    appointment_date = '2024-10-01',
    appointment_time = '11:00',
)

In [9]:
# Check result
res = Appointment.objects.last()
print(f"Appoint Date: {res.appointment_date}, Appoint Time: {res.appointment_time}")
print(f"Service Name: {res.service.name}, Provider Name: {res.service.service_provider.name}, Customer Name: {res.customer.name}")

Appoint Date: 2024-10-01, Appoint Time: 11:00:00
Service Name: Haircut, Provider Name: Provider 2, Customer Name: Customer 1


1.2 ทำการแก้ไขนัดหมายที่เพิ่งสร้างใน 1.1 โดยเปลี่ยนวันที่จากวันที่ 2024-10-01 เป็น 2024-10-10 เวลาเดิม

(0.25 คะแนน)

In [10]:
res = Appointment.objects.last()
res.appointment_date = '2024-10-10'
res.save()

In [11]:
# Check result
res = Appointment.objects.last()
print(f"Appoint Date: {res.appointment_date}, Appoint Time: {res.appointment_time}")
print(f"Service Name: {res.service.name}, Provider Name: {res.service.service_provider.name}, Customer Name: {res.customer.name}")

Appoint Date: 2024-10-10, Appoint Time: 11:00:00
Service Name: Haircut, Provider Name: Provider 2, Customer Name: Customer 1


1.3 ทำการแก้ไขนัดหมาย ณ วันที่ 2024-08-14 ของ "Customer 4" โดยเปลี่ยนจากเดิมที่นัดมาให้บริการ "Manicure" ของ "Provider 3" เป็น "Massage" ของ "Provider 4" แทน

(0.25 คะแนน)

*หมายเหตุ: จะต้อง get ข้อมูลมาโดยใช้การ query ด้วยชื่อตามที่โจทย์ว่าเท่านั้น ห้ามใช้ id ใน database*

In [20]:
massage = Service.objects.get(service_provider__name = 'Provider 4', name = 'Massage')

appointment = Appointment.objects.get(
    customer__name = 'Customer 4',
    appointment_date = '2024-08-14'
)

appointment.service = massage
appointment.save()

In [24]:
# Check result
res = Appointment.objects.get(customer__name="Customer 4", appointment_date=date(2024,8,14))
print(f"Appoint Date: {res.appointment_date}, Appoint Time: {res.appointment_time}")
print(f"Service Name: {res.service.name}, Provider Name: {res.service.service_provider.name}, Customer Name: {res.customer.name}")

Appoint Date: 2024-08-14, Appoint Time: 10:00:00
Service Name: Massage, Provider Name: Provider 4, Customer Name: Customer 4


1.4 ทำตามขั้นตอนดังนี้

1. สร้างหมวดหมู่บริการ (`ServiceCategory`) ใหม่ชื่อ "Facial & Body Care" และมีรายละเอียดว่า "Services related to face & body care" 
2. สร้าง บริการ (`Service`) ใหม่ดังนี้ 

    - Name: "Face Massage"
    - Description: "Special face massage!"
    - Price: 150.00
    - โดยบริการนี้ให้บริการโดย "Provider 1" และ "Provider 2" (Hint: ดังนั้นต้องสร้าง 2 รายการ)

3. เพิ่มบริการใหม่ทั้ง 2 รายการเข้าไปในหมวดหมู่ "Facial & Body Care"
4. ทำการย้ายบริการ (`Service`) ในหมวดหมู่บริการ (`ServiceCategory`) "Body Care" ไปที่ หมวดหมู่ "Facial & Body Care" ที่สร้างขึ้นมาใหม่
5. ทำการลบหมวดหมู่บริการ "Body Care"

(1 คะแนน)

In [25]:
# 1. 
facial = ServiceCategory.objects.create(
    name = "Facial & Body Care", 
    description = "Services related to face & body care",
)

In [31]:
# 2. + 3
facial = ServiceCategory.objects.get(name = "Facial & Body Care")
provider_1 = ServiceProvider.objects.get(name = 'Provider 1')
provider_2 = ServiceProvider.objects.get(name = 'Provider 2')

face_1 = Service.objects.create(
    service_provider = provider_1,
    name = "Face Massage",
    description =  "Special face massage!",
    price = 150.00
)
facial.services.add(face_1 )

face_2 = Service.objects.create(
    service_provider = provider_2,
    name = "Face Massage",
    description =  "Special face massage!",
    price = 150.00
)
facial.services.add(face_2)

In [66]:
# 4.
facial = ServiceCategory.objects.get(name = "Facial & Body Care")
body = ServiceCategory.objects.get(name = "Body Care")
body_service = Service.objects.filter(servicecategory = body)

for service in body_service:
    service.servicecategory_set.remove(body)
    service.servicecategory_set.add(facial)

In [ ]:
# 5.
body = ServiceCategory.objects.get(name = "Body Care")
body.delete()

In [50]:
# Check result
sc1 = ServiceCategory.objects.get(name="Facial & Body Care")
sc1.services.count()

7

## PART 2 - Making Queries

สำหรับ PART 2 ให้ทำการ reset DB และ import ข้อมูลใน `service.sql` เข้าไปใหม่

2.1 ให้หาจำนวนนัดหมาย ที่ถูกสร้าง (`Appointment.created_at`) ในช่วงวันที่ 2024-08-01 ถึง 2024-08-02 

(0.5 คะแนน)

In [26]:
count_app = Appointment.objects.aggregate(count = Count("created_at" , 
                                                        filter=Q(created_at__range = 
                                                                 (datetime(2024,8,1), datetime(2024,8,2))) 
                                                       )
                                         )
print(count_app)

{'count': 2}


2.2 ให้หาว่ามีบริการใด ของผู้ให้บริการไหนบ้าง ที่มีการนัดหมายท่ีมีการนัดหมาย (`Appointment.appoint_date`) ในช่วงวันที่ 2024-08-10 ถึง 2024-08-15

(0.5 คะแนน)

**Expected Output**

```
Name: Haircut, Provider: Provider 1
Name: Massage, Provider: Provider 1
Name: Massage, Provider: Provider 2
Name: Massage, Provider: Provider 3
Name: Massage, Provider: Provider 4
```

In [33]:
app_date = Appointment.objects.filter(appointment_date__range = (datetime(2024,8,10), datetime(2024,8,15))) 

for app in app_date:
    print(f"Name: {app.service.name}, Provider: {app.service.service_provider.name}")


Name: Manicure, Provider: Provider 2
Name: Massage, Provider: Provider 2
Name: Manicure, Provider: Provider 2
Name: Manicure, Provider: Provider 3
Name: Massage, Provider: Provider 3
Name: Massage, Provider: Provider 1
Name: Manicure, Provider: Provider 2


2.3 ให้แสดงผลข้อมูลบริการ (`Service`) โดยเรียงลำดับตามราคา (`Service.price`) จากน้อยไปมาก ดังในตัวอย่าง

```python
[
    {
        "price": 100,
        "full_name": "Massage (Provider 1)"
    },
    {
        "price": 100,
        "full_name": "Massage (Provider 2)"
    },
    {
        "price": 100,
        "full_name": "Massage (Provider 3)"
    },
    {
        "price": 100,
        "full_name": "Massage (Provider 4)"
    },
    {
        "price": 30,
        "full_name": "Manicure (Provider 4)"
    },
    {
        "price": 30,
        "full_name": "Manicure (Provider 1)"
    },
    {
        "price": 30,
        "full_name": "Manicure (Provider 3)"
    },
    {
        "price": 30,
        "full_name": "Manicure (Provider 2)"
    },
    {
        "price": 20,
        "full_name": "Haircut (Provider 4)"
    },
    {
        "price": 20,
        "full_name": "Haircut (Provider 2)"
    },
    {
        "price": 20,
        "full_name": "Haircut (Provider 3)"
    },
    {
        "price": 20,
        "full_name": "Haircut (Provider 1)"
    }
]
```

(0.5 คะแนน)

*หมายเหตุ: จะต้องใช้ `annotate()` และ `values()` เท่านั้น การ loop และสร้าง dict เองจะไม่ได้คะแนน*

Hint: อยาก print dictionary สวยๆ ใช้ json.dumps

```python
print(json.dumps(dictionary, indent=4, sort_keys=False))
```

In [39]:
service = Service.objects.annotate( full_name = 
                                   Concat(F('name') , Value(' (') , F('service_provider__name'), Value(')') 
                                         )
                                  ).order_by('price').values('price', 'full_name')
services = list(service)
print(json.dumps(services, indent=4, sort_keys=False))

[
    {
        "price": 20,
        "full_name": "Haircut (Provider 1)"
    },
    {
        "price": 20,
        "full_name": "Haircut (Provider 2)"
    },
    {
        "price": 20,
        "full_name": "Haircut (Provider 3)"
    },
    {
        "price": 20,
        "full_name": "Haircut (Provider 4)"
    },
    {
        "price": 30,
        "full_name": "Manicure (Provider 4)"
    },
    {
        "price": 30,
        "full_name": "Manicure (Provider 1)"
    },
    {
        "price": 30,
        "full_name": "Manicure (Provider 3)"
    },
    {
        "price": 30,
        "full_name": "Manicure (Provider 2)"
    },
    {
        "price": 100,
        "full_name": "Massage (Provider 4)"
    },
    {
        "price": 100,
        "full_name": "Massage (Provider 2)"
    },
    {
        "price": 100,
        "full_name": "Massage (Provider 3)"
    },
    {
        "price": 100,
        "full_name": "Massage (Provider 1)"
    }
]


2.4 ให้หาจำนวนนัดหมายของลูกค้าแต่ละคน

(0.5 คะแนน)

*หมายเหตุ: จะต้องใช้ `annotate()` และ `values()`*

**Expected Output**

```python
Name: Customer 4, Total Appoinment: 3
Name: Customer 2, Total Appoinment: 4
Name: Customer 3, Total Appoinment: 3
Name: Customer 1, Total Appoinment: 5
```

In [1]:
app_count = Customer.objects.annotate(
    customer_count = Count('appointment')
).values('name', 'customer_count')
# .annotate แล้ว .values ต่อ คือการ SELECT เฉพาะข้อมูลใน SQL และทำการ GROUP BY 
# ด้วย id ของตารางมันเองต้องกำหนด GROUP ดีๆ แต่ถ้า values ก่อนจะเป็นการ GROUP BY 

for app in app_count:
    print(f"Name: {app['name']}, Total Appoinment: {app['customer_count']}")

SELECT "service_customer"."name", COUNT("service_appointment"."id") AS "customer_count" FROM "service_customer" LEFT OUTER JOIN "service_appointment" ON ("service_customer"."id" = "service_appointment"."customer_id") GROUP BY "service_customer"."id"


SynchronousOnlyOperation: You cannot call this from an async context - use a thread or sync_to_async.

2.5 ให้หาว่าผู้ให้บริการ (`ServiceProvider`) แต่ละคนนั้นได้เงินโดยรวมเท่าไหร่

Hint: ดูจากนั้ดหมายว่ามีการนัดบริการ (`Appointment.service`) ของแต่ละผู้ให้บริการรวมเป็นยอดเงินเท่าไหร่

*หมายเหตุ: ไม่จำเป็นต้องใช้ `annotate()` สามารถ loop ได้เต็มที่เลยครับ*

**Expected Output**

```python
Name: Provider 4, Total Price: 20
Name: Provider 2, Total Price: 210
Name: Provider 3, Total Price: 130
Name: Provider 1, Total Price: 410
```

In [5]:
sum_pro = ServiceProvider.objects.annotate(
    sum_provider = Sum('service__price')
)

for pro in sum_pro:
    print(f'Name: Provider {pro.name}, Total Price: {pro.sum_provider}')

Name: Provider Provider 4, Total Price: 150
Name: Provider Provider 2, Total Price: 150
Name: Provider Provider 3, Total Price: 150
Name: Provider Provider 1, Total Price: 150


2.6 ให้ดึงข้อมูลหมวดหมู่บริการโดยแปลงชื่อหมวดหมู่บริการ (`ServiceCategory.name`) เป็นตัวใหญ่ (uppercase) ทั้งหมด

(0.5 คะแนน)

**Expected Output**

```python
Name: Hair Care, Upper Name: HAIR CARE
Name: Nail Care, Upper Name: NAIL CARE
Name: Body Care, Upper Name: BODY CARE
```

In [3]:
cat_up = ServiceCategory.objects.all()

for cat in cat_up:
    print(f'Name: {cat.name}, Upper Name: {cat.name.upper()}')

Name: Hair Care, Upper Name: HAIR CARE
Name: Nail Care, Upper Name: NAIL CARE
Name: Body Care, Upper Name: BODY CARE
